In [1]:
import copy
import gc
import io
import json
import logging
import os
import random
import shutil
import sys
import time
from typing_extensions import Any, Dict, List, Tuple, Union

from sklearnex import patch_sklearn
patch_sklearn()

import joblib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
from fvcore.nn import FlopCountAnalysis
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from torchsummary import summary
from tqdm_loggable.auto import tqdm

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)
/home/b23cs1037/MLOps-Laksh-B23CS1037/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TqdmToLogger(io.StringIO):
    def __init__(self, logger, level=logging.INFO):
        super().__init__()
        self.logger = logger
        self.level = level

    def write(self, buf):
        # strip() removes the \r and \n characters tqdm uses for terminal updates
        content = buf.strip('\r\n\t ')
        if content:
            self.logger.log(self.level, content)

    def flush(self):
        pass

In [3]:
def get_logger():
    logger = logging.getLogger("main_logger")
    logger.setLevel(logging.INFO)
    
    # Clear existing handlers if any (prevents duplicate logs in notebooks)
    if logger.hasHandlers():
        logger.handlers.clear()

    # Create a format for your logs
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Stream Handler (Standard Output)
    sh = logging.StreamHandler(sys.stdout)
    sh.setFormatter(formatter)
    logger.addHandler(sh)

    # File Handler (Direct logging to a file)
    fh = logging.FileHandler("experiment_progress.log")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger


logger = get_logger()
tqdm_out = TqdmToLogger(logger, level=logging.INFO)

In [4]:
SEED = 42
generator = torch.Generator()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

def set_seed(seed: int = SEED) -> None:
    # set random seed for reproducibility
    logger.info(f"Setting seed: {seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    generator.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

2026-01-24 01:36:02,787 - INFO - Using device: cuda:0


2026-01-24 01:36:02,789 - INFO - Setting seed: 42


In [5]:
mnist_dataset = None
fashion_mnist_dataset = None
train_split_ratio = 0.7
val_split_ratio = 0.1
test_split_ratio = 0.2

In [6]:
def load_mnist_dataset() -> None:
    logger.info("Loading MNIST dataset...")
    global mnist_dataset
    
    if mnist_dataset is None:
        train_dataset = torchvision.datasets.MNIST(
            root='./data',
            train=True,
            transform=torchvision.transforms.ToTensor(),
            download=True
        )
        test_dataset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            transform=torchvision.transforms.ToTensor(),
            download=True
        )
        
        # concatenate train and test datasets
        mnist_dataset = torch.utils.data.ConcatDataset(
            [
                train_dataset,
                test_dataset
            ]
        )
        
        # split into train, val, test
        total_size = len(mnist_dataset)
        train_size = int(train_split_ratio * total_size)
        val_size = int(val_split_ratio * total_size)
        test_size = total_size - train_size - val_size
        mnist_train, mnist_val, mnist_test = torch.utils.data.random_split(
            mnist_dataset,
            [train_size, val_size, test_size],
            generator=generator
        )
        
        mnist_dataset = {
            "train": mnist_train,
            "val": mnist_val,
            "test": mnist_test
        }
        
        del train_dataset, test_dataset, mnist_train, mnist_val, mnist_test


def load_fashion_mnist_dataset() -> None:
    logger.info("Loading Fashion-MNIST dataset...")
    global fashion_mnist_dataset
    
    if fashion_mnist_dataset is None:
        train_dataset = torchvision.datasets.FashionMNIST(
            root='./data',
            train=True,
            transform=torchvision.transforms.ToTensor(),
            download=True
        )
        test_dataset = torchvision.datasets.FashionMNIST(
            root='./data',
            train=False,
            transform=torchvision.transforms.ToTensor(),
            download=True
        )
        # concatenate train and test datasets
        fashion_mnist_dataset = torch.utils.data.ConcatDataset(
            [
                train_dataset,
                test_dataset
            ]
        )
        # split into train, val, test
        total_size = len(fashion_mnist_dataset)
        train_size = int(train_split_ratio * total_size)
        val_size = int(val_split_ratio * total_size)
        test_size = total_size - train_size - val_size
        fashion_mnist_train, fashion_mnist_val, fashion_mnist_test = torch.utils.data.random_split(
            fashion_mnist_dataset,
            [train_size, val_size, test_size],
            generator=generator
        )
        
        fashion_mnist_dataset = {
            "train": fashion_mnist_train,
            "val": fashion_mnist_val,
            "test": fashion_mnist_test
        }
        
        del train_dataset, test_dataset, fashion_mnist_train, fashion_mnist_val, fashion_mnist_test

def load_datasets() -> None:
    load_mnist_dataset()
    load_fashion_mnist_dataset()

In [7]:
load_datasets()

2026-01-24 01:36:02,909 - INFO - Loading MNIST dataset...


2026-01-24 01:36:03,027 - INFO - Loading Fashion-MNIST dataset...


In [8]:
# dataset information
logger.info(f"MNIST train data size - {len(mnist_dataset['train'])}")
logger.info(f"MNIST train data size - {len(mnist_dataset['val'])}")
logger.info(f"MNIST train data size - {len(mnist_dataset['test'])}")
logger.info(f"MNIST image shape - {mnist_dataset['train'][0][0].shape}\n")

logger.info(f"Fashion-MNIST train data size - {len(fashion_mnist_dataset['train'])}")
logger.info(f"Fashion-MNIST train data size - {len(fashion_mnist_dataset['val'])}")
logger.info(f"Fashion-MNIST train data size - {len(fashion_mnist_dataset['test'])}")
logger.info(f"Fashion-MNIST image shape - {fashion_mnist_dataset['train'][0][0].shape}")

2026-01-24 01:36:03,156 - INFO - MNIST train data size - 49000


2026-01-24 01:36:03,157 - INFO - MNIST train data size - 7000


2026-01-24 01:36:03,158 - INFO - MNIST train data size - 14000


2026-01-24 01:36:03,160 - INFO - MNIST image shape - torch.Size([1, 28, 28])



2026-01-24 01:36:03,161 - INFO - Fashion-MNIST train data size - 49000


2026-01-24 01:36:03,162 - INFO - Fashion-MNIST train data size - 7000


2026-01-24 01:36:03,163 - INFO - Fashion-MNIST train data size - 14000


2026-01-24 01:36:03,164 - INFO - Fashion-MNIST image shape - torch.Size([1, 28, 28])


In [9]:
train_transform = T.Compose([
    T.ToPILImage(),
    T.Resize(256),
    T.RandomResizedCrop(
        224,
        scale=(0.8, 1.0),
        ratio=(0.9, 1.1)
    ),
    T.RandomRotation(15),
    T.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1)
    ),
    T.Grayscale(num_output_channels=3),  # 3 channel for resnet
    T.ToTensor(),
    T.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])


test_transform = T.Compose([
    T.ToPILImage(),
    T.Resize(224),
    T.Grayscale(num_output_channels=3),
    T.ToTensor(),
    T.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [10]:
class customDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        base_dataset: torch.utils.data.Dataset,
        transform: Union[torchvision.transforms.Compose, None] = None
    ):
        self.base_dataset = base_dataset
        self.transform = transform
    
    def __len__(self) -> int:
        return len(self.base_dataset)
    
    def __getitem__(self, idx: int) -> Any:
        img, label = self.base_dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [11]:
aug_mnist_dataset = None
aug_fashion_mnist_dataset = None

def load_augmented_datasets() -> None:
    logger.info("Loading augmented datasets...")
    global aug_mnist_dataset, aug_fashion_mnist_dataset
    
    if aug_mnist_dataset is None:
        aug_mnist_dataset = {
            "train": customDataset(
                mnist_dataset["train"],
                transform=train_transform
            ),
            "val": customDataset(
                mnist_dataset["val"],
                transform=test_transform
            ),
            "test": customDataset(
                mnist_dataset["test"],
                transform=test_transform
            )
        }
    
    if aug_fashion_mnist_dataset is None:
        aug_fashion_mnist_dataset = {
            "train": customDataset(
                fashion_mnist_dataset["train"],
                transform=train_transform
            ),
            "val": customDataset(
                fashion_mnist_dataset["val"],
                transform=test_transform
            ),
            "test": customDataset(
                fashion_mnist_dataset["test"],
                transform=test_transform
            )
        }

load_augmented_datasets()

2026-01-24 01:36:03,262 - INFO - Loading augmented datasets...


In [12]:
# dataset information
logger.info(f"MNIST train data size - {len(aug_mnist_dataset['train'])}")
logger.info(f"MNIST train data size - {len(aug_mnist_dataset['val'])}")
logger.info(f"MNIST train data size - {len(aug_mnist_dataset['test'])}")
logger.info(f"MNIST image shape - {aug_mnist_dataset['train'][0][0].shape}\n")

logger.info(f"Fashion-MNIST train data size - {len(aug_fashion_mnist_dataset['train'])}")
logger.info(f"Fashion-MNIST train data size - {len(aug_fashion_mnist_dataset['val'])}")
logger.info(f"Fashion-MNIST train data size - {len(aug_fashion_mnist_dataset['test'])}")
logger.info(f"Fashion-MNIST image shape - {aug_fashion_mnist_dataset['train'][0][0].shape}")

2026-01-24 01:36:03,288 - INFO - MNIST train data size - 49000


2026-01-24 01:36:03,289 - INFO - MNIST train data size - 7000


2026-01-24 01:36:03,290 - INFO - MNIST train data size - 14000


2026-01-24 01:36:03,303 - INFO - MNIST image shape - torch.Size([3, 224, 224])



2026-01-24 01:36:03,304 - INFO - Fashion-MNIST train data size - 49000


2026-01-24 01:36:03,305 - INFO - Fashion-MNIST train data size - 7000


2026-01-24 01:36:03,305 - INFO - Fashion-MNIST train data size - 14000


2026-01-24 01:36:03,310 - INFO - Fashion-MNIST image shape - torch.Size([3, 224, 224])


In [13]:
def get_resnet18_model(num_classes: int = 10) -> torch.nn.Module:
    model = torchvision.models.resnet18(pretrained=False)
    # Modify the final layer to match num_classes
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

def get_resnet32_model(num_classes: int = 10) -> torch.nn.Module:
    model = torchvision.models.resnet34(pretrained=False)
    # Modify the final layer to match num_classes
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

def get_resnet50_model(num_classes: int = 10) -> torch.nn.Module:
    model = torchvision.models.resnet50(pretrained=False)
    # Modify the final layer to match num_classes
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model

In [14]:
logger.info("ResNet18 Model Summary:")
summary(get_resnet18_model().to(device), (3, 224, 224)) # ~11 million parameters

2026-01-24 01:36:03,362 - INFO - ResNet18 Model Summary:


/home/b23cs1037/MLOps-Laksh-B23CS1037/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/b23cs1037/MLOps-Laksh-B23CS1037/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-1                   [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-2                   [-1, 64, 56, 56]          --
|

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-1                   [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-2                   [-1, 64, 56, 56]          --
|

In [15]:
logger.info("ResNet32 Model Summary:")
summary(get_resnet32_model().to(device), (3, 224, 224)) # ~21 million parameters

2026-01-24 01:36:04,274 - INFO - ResNet32 Model Summary:


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-1                   [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-2                   [-1, 64, 56, 56]          --
|

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-1                   [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    └─BasicBlock: 2-2                   [-1, 64, 56, 56]          --
|

In [16]:
logger.info("ResNet50 Model Summary:")
summary(get_resnet50_model().to(device), (3, 224, 224)) # ~25 million parameters

2026-01-24 01:36:04,725 - INFO - ResNet50 Model Summary:


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-1                   [-1, 256, 56, 56]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          4,096
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-7                  [-1, 256, 56, 56]         16,38

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 112, 112]        9,408
├─BatchNorm2d: 1-2                       [-1, 64, 112, 112]        128
├─ReLU: 1-3                              [-1, 64, 112, 112]        --
├─MaxPool2d: 1-4                         [-1, 64, 56, 56]          --
├─Sequential: 1-5                        [-1, 256, 56, 56]         --
|    └─Bottleneck: 2-1                   [-1, 256, 56, 56]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 56, 56]          4,096
|    |    └─BatchNorm2d: 3-2             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-3                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-4                  [-1, 64, 56, 56]          36,864
|    |    └─BatchNorm2d: 3-5             [-1, 64, 56, 56]          128
|    |    └─ReLU: 3-6                    [-1, 64, 56, 56]          --
|    |    └─Conv2d: 3-7                  [-1, 256, 56, 56]         16,38

### Q1, a) Training RESNET-18, 50 on MNIST, Fashion-MNIST

In [17]:
def get_data_loader(
    dataset: torch.utils.data.Dataset,
    batch_size: int,
    drop_last: bool = False,
    shuffle: bool = True,
    num_workers: int = 8,
    pin_memory: bool = True,
    persistent_workers:bool =True,
) -> torch.utils.data.DataLoader:
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=persistent_workers,
        generator=generator
    )

In [18]:
# calculate loss and accuracy
def calculate_loss_and_accuracy(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    device: torch.device
) -> Dict[str, float]:
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += inputs.size(0)
    
    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    
    return {
        "loss": avg_loss,
        "accuracy": accuracy
    }

In [19]:
def save_training_plots(
    training_stats: Dict[str, list],
    save_dir: str
) -> None:
    epochs = range(1, len(training_stats["train_loss"]) + 1)
    
    # Plot Loss
    # x-axis should be intiger
    
    plt.figure()
    plt.plot(epochs, training_stats["train_loss"], label='Train Loss')
    plt.plot(epochs, training_stats["val_loss"], label='Validation Loss')
    # plt.plot(epochs, training_stats["test_loss"], label='Test Loss')
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()
    plt.grid(True)
    loss_plot_path = os.path.join(save_dir, 'loss_plot.png')
    plt.savefig(loss_plot_path)
    plt.close()
    
    # Plot Accuracy
    plt.figure()
    plt.plot(epochs, training_stats["train_accuracy"], label='Train Accuracy')
    plt.plot(epochs, training_stats["val_accuracy"], label='Validation Accuracy')
    # plt.plot(epochs, training_stats["test_accuracy"], label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()
    plt.grid(True)
    acc_plot_path = os.path.join(save_dir, 'accuracy_plot.png')
    plt.savefig(acc_plot_path)
    plt.close()

In [20]:
def save_model_and_stats(
    model: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    config: Dict[str, Any],
    training_stats: Dict[str, list],
    base_dir: str    
) -> Tuple[str, str, str]:
    # model_save_dir_name = (
    #     f"model_{config['dataset_name']}_{config['model_name']}_{config['optimizer_name']}_",
    #     f"lr_{str(config['learning_rate']).replace('.', '_')}_batch{config['batch_size']}_epoch{epoch+1}"
    # )
    model_save_dir_name = '_'.join(f'{key}_{value}' for key, value in config.items())
    model_dir = os.path.join(base_dir, ''.join(model_save_dir_name))
    os.makedirs(model_dir, exist_ok=True) 
    
    model_path = os.path.join(model_dir, 'model.pth')
    model_save_state = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'training_stats': training_stats,
        'config': config
    }
    torch.save(model_save_state, model_path)
    
    del model_save_state
    return model_path, model_dir, base_dir

In [21]:
# @torch.compile(fullgraph=True)
def train_model(config: Dict[str, Any], base_dir: str, device:torch.device) -> List[Dict[str, Any]]:
    
    is_cuda = device.type == "cuda"
    torch.set_float32_matmul_precision('high')

    # config has keys: dataset_name, model_name, learning_rate, batch_size, optimizer_name, num_epochs
    if config["model_name"] == "resnet18":
        model = get_resnet18_model().to(device)
    elif config["model_name"] == "resnet32":
        model = get_resnet32_model().to(device)
    elif config["model_name"] == "resnet50":
        model = get_resnet50_model().to(device)
    else:
        raise ValueError(f"Unsupported model name: {config['model_name']}")

    # model = torch.compile(model)
    if is_cuda:
        model = torch.compile(model, fullgraph=True)
    else:
        model = torch.compile(model, backend='inductor', dynamic=True)
        
    
    if config["optimizer_name"] == "sgd":
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config["learning_rate"],
            momentum=0.9
        )
    elif config["optimizer_name"] == "adam":
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config["learning_rate"]
        )
    else:
        raise ValueError(f"Unsupported optimizer name: {config['optimizer_name']}")
    
    dl_kwargs = {
        "num_workers": min(8, os.cpu_count() or 0), 
        "pin_memory": False if not is_cuda else True,
        "persistent_workers": True if (os.cpu_count() or 0) > 0 else False
    }
    
    train_data_loader = get_data_loader(
        aug_mnist_dataset["train"] if config["dataset_name"] == "mnist" else aug_fashion_mnist_dataset["train"],
        batch_size=config["batch_size"],
        shuffle=True,
        drop_last=True,
        **dl_kwargs
    )
    
    val_data_loader = get_data_loader(
        aug_mnist_dataset["val"] if config["dataset_name"] == "mnist" else aug_fashion_mnist_dataset["val"],
        batch_size=config["batch_size"],
        shuffle=False,
        drop_last=False,
        **dl_kwargs
    )
    
    test_data_loader = get_data_loader(
        aug_mnist_dataset["test"] if config["dataset_name"] == "mnist" else aug_fashion_mnist_dataset["test"],
        batch_size=config["batch_size"],
        shuffle=False,
        drop_last=False,
        **dl_kwargs
    )
    
    training_stats = {
        "train_loss": [],
        "train_accuracy": [],
        "val_loss": [],
        "val_accuracy": []
    }
    
    scaler = torch.amp.GradScaler(device=device.type, enabled=is_cuda)
    
    autocast_kwargs = {
        "device_type": "cuda" if is_cuda else "cpu",
        "dtype": torch.float16 if is_cuda else torch.bfloat16
    }
    
    criterion = torch.nn.CrossEntropyLoss()
    
    start_time = time.time()
    return_data = []
    
    for epoch in range(config["num_epochs"]):

        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in tqdm(
            train_data_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}", file=tqdm_out,
            mininterval=30.0
        ):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad(set_to_none=True)

            with torch.autocast(**autocast_kwargs):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            if is_cuda:
                scaler.scale(loss).backward()
                # loss.backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs.float(), 1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += inputs.size(0)
            
        epoch_loss = running_loss / total_samples
        epoch_accuracy = correct_predictions / total_samples
        
        with torch.no_grad():
            val_metrics = calculate_loss_and_accuracy(
                model,
                val_data_loader,
                criterion,
                device
            )

        model.train()
        
        # test_metrics = calculate_loss_and_accuracy(
        #     model,
        #     test_data_loader,
        #     criterion,
        #     device
        # )
        
        training_stats["train_loss"].append(epoch_loss)
        training_stats["train_accuracy"].append(epoch_accuracy)
        training_stats["val_loss"].append(val_metrics["loss"])
        training_stats["val_accuracy"].append(val_metrics["accuracy"])
        # training_stats["test_loss"].append(test_metrics["loss"])
        # training_stats["test_accuracy"].append(test_metrics["accuracy"])
        
        # if (epoch + 1) % 10 == 0:
        logger.info(
            f"Epoch [{epoch+1}/{config['num_epochs']}], "
            f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_accuracy:.4f}, "
            f"Val Loss: {val_metrics['loss']:.4f}, Val Acc: {val_metrics['accuracy']:.4f}, "
            # f"Test Loss: {test_metrics['loss']:.4f}, Test Acc: {test_metrics['accuracy']:.4f}"
        )
        
        # save intermediate model at every 5 epochs or final epoch
        if (epoch + 1)%5==0 or epoch+1==config["num_epochs"]:
            mid_train_time = time.time()
            mid_elapsed_time = mid_train_time - start_time
            logger.info(f"Mid Training Time after {epoch+1} epochs: {mid_elapsed_time/60:.2f} minutes.")
            
            mid_stats = copy.deepcopy(training_stats)
            mid_config = copy.deepcopy(config)
            mid_config["num_epochs"] = epoch + 1
            
            mid_model_path, mid_model_dir, _ = save_model_and_stats(
                model,
                optimizer,
                mid_config,
                mid_stats,
                base_dir=base_dir
            )
            save_training_plots(mid_stats, mid_model_dir)
            logger.info(f"Mid-training model saved at: {mid_model_path}")
            
            with torch.no_grad():
                mid_test_metrics = calculate_loss_and_accuracy(
                    model,
                    test_data_loader,
                    criterion,
                    device
                )
            model.train()
            
            mid_stats["model_path"] = mid_model_path
            mid_stats["model_dir"] = mid_model_dir
            mid_stats["base_dir"] = base_dir
            mid_stats["training_time"] = mid_elapsed_time # in seconds
            mid_stats["test_loss"] = mid_test_metrics["loss"]
            mid_stats["test_accuracy"] = mid_test_metrics["accuracy"]
            
            return_data.append({**mid_config, **mid_stats})
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    logger.info(f"Training completed in {elapsed_time/60:.2f} minutes.")
    
    del model, optimizer, train_data_loader, val_data_loader, test_data_loader
    torch.cuda.empty_cache()
    gc.collect()
    return return_data

In [22]:
# to search for both mnist and fashion-mnist datasets for every configuration
base_experiment_dir = './q1_a_experiments'
os.makedirs(base_experiment_dir, exist_ok=True)

mnist_search_space = {
    "dataset_name": "mnist",
    "model_name": ["resnet18", "resnet50"],
    "learning_rate": [0.001, 0.0001],
    "batch_size": [16, 32],
    "optimizer_name": ["sgd", "adam"],
    "num_epochs": [10]
}

results = []

for model_name in mnist_search_space["model_name"]:
    for learning_rate in mnist_search_space["learning_rate"]:
        for batch_size in mnist_search_space["batch_size"]:
            for optimizer_name in mnist_search_space["optimizer_name"]:
                for num_epochs in mnist_search_space["num_epochs"]:
                    config = {
                        "dataset_name": mnist_search_space["dataset_name"],
                        "model_name": model_name,
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "optimizer_name": optimizer_name,
                        "num_epochs": num_epochs
                    }
                    logger.info(f"Training with config: {config}")
                    return_data = train_model(
                        config, os.path.join(base_experiment_dir, 'mnist_experiments'),
                        device=device
                    )
                    logger.info(f"Training completed.\n")
                    results.extend(return_data)

# save the results to json file
result_path = os.path.join(base_experiment_dir, 'mnist_experiments_results.json')
with open(result_path, 'w') as f:
    json.dump(results, f, indent=4)

# identify best model based on test accuracy
best_model = max(results, key=lambda x: x['test_accuracy'])
logger.info(f"Best Model Config: {best_model}")
logger.info(f"Best Model Test Accuracy: {best_model['test_accuracy']:.4f}")

# save best model to base_dir/mnist_best_model.pth
best_model_src_path = best_model['model_path']
best_model_dest_path = os.path.join(base_experiment_dir, 'mnist_best_model.pth')
shutil.copy(best_model_src_path, best_model_dest_path)
logger.info(f"Best model saved to: {best_model_dest_path}")

2026-01-24 01:36:05,523 - INFO - Training with config: {'dataset_name': 'mnist', 'model_name': 'resnet18', 'learning_rate': 0.001, 'batch_size': 16, 'optimizer_name': 'sgd', 'num_epochs': 10}


2026-01-24 01:36:06,174 - INFO - Epoch 1/10:   0%|          | 0/3062 [00:00<?, ?it/s]


2026-01-24 01:36:36,202 - INFO - Epoch 1/10:  27%|##6       | 821/3062 [00:30<01:21, 27.34it/s]


2026-01-24 01:36:50,205 - INFO - Epoch 1/10:  27%|##6       | 821/3062 [00:44<01:21, 27.34it/s]


2026-01-24 01:37:06,214 - INFO - Epoch 1/10:  60%|#####9    | 1823/3062 [01:00<00:40, 30.90it/s]


...

2026-01-24 06:40:02,661 - INFO - Training completed in 19.48 minutes.


2026-01-24 06:40:03,517 - INFO - Training completed.



2026-01-24 06:40:03,521 - INFO - Best Model Config: {'dataset_name': 'mnist', 'model_name': 'resnet18', 'learning_rate': 0.0001, 'batch_size': 16, 'optimizer_name': 'adam', 'num_epochs': 10, 'train_loss': [0.1633802874927463, 0.060455049299905694, 0.04358551813779821, 0.039053830665497904, 0.03135669265480154, 0.032394958865615296, 0.026603072180925516, 0.024568772146625506, 0.022374051946058997, 0.02155458061407782], 'train_accuracy': [0.9526657413455258, 0.9817521227955585, 0.9869978772044415, 0.9879776290006531, 0.9907944154147615, 0.9900187785760941, 0.9918354016982365, 0.9921823971260614, 0.9932029719137818, 0.9931213259307642], 'val_loss': [0.06926200069713273, 0.028877460737380067, 0.029239134590374305, 0.02574423217133153, 0.026651678801281376, 0.02183751707727788, 0.02461808457067569, 0.021685700012404205, 0.021550163860682266, 0.016939437369445972], 'val_accuracy': [0.9784285714285714, 0.9918571428571429, 0.9915714285714285, 0.9918571428571429, 0.9931428571428571, 0.993142857

2026-01-24 06:40:03,522 - INFO - Best Model Test Accuracy: 0.9942


2026-01-24 06:40:03,597 - INFO - Best model saved to: ./q1_a_experiments/mnist_best_model.pth


In [23]:
fashion_mnist_search_space = {
    "dataset_name": "fashion-mnist",
    "model_name": ["resnet18", "resnet50"],
    "learning_rate": [0.001, 0.0001],
    "batch_size": [16, 32],
    "optimizer_name": ["sgd", "adam"],
    "num_epochs": [10]
}

results = []

for model_name in fashion_mnist_search_space["model_name"]:
    for learning_rate in fashion_mnist_search_space["learning_rate"]:
        for batch_size in fashion_mnist_search_space["batch_size"]:
            for optimizer_name in fashion_mnist_search_space["optimizer_name"]:
                for num_epochs in fashion_mnist_search_space["num_epochs"]:
                    config = {
                        "dataset_name": fashion_mnist_search_space["dataset_name"],
                        "model_name": model_name,
                        "learning_rate": learning_rate,
                        "batch_size": batch_size,
                        "optimizer_name": optimizer_name,
                        "num_epochs": num_epochs
                    }
                    logger.info(f"Training with config: {config}")
                    return_data = train_model(
                        config, os.path.join(base_experiment_dir, 'fashion_mnist_experiments'),
                        device=device
                    )
                    logger.info(f"Training completed.\n")
                    results.extend(return_data)

# save the results to json file
result_path = os.path.join(base_experiment_dir, 'fashion_mnist_experiments_results.json')
with open(result_path, 'w') as f:
    json.dump(results, f, indent=4)

# identify best model based on test accuracy
best_model = max(results, key=lambda x: x['test_accuracy'])
logger.info(f"Best Model Config: {best_model}")
logger.info(f"Best Model Test Accuracy: {best_model['test_accuracy']:.4f}")

# save best model to base_dir/mnist_best_model.pth
best_model_src_path = best_model['model_path']
best_model_dest_path = os.path.join(base_experiment_dir, 'fashion_mnist_best_model.pth')
shutil.copy(best_model_src_path, best_model_dest_path)
logger.info(f"Best model saved to: {best_model_dest_path}")

2026-01-24 06:40:03,817 - INFO - Training with config: {'dataset_name': 'fashion-mnist', 'model_name': 'resnet18', 'learning_rate': 0.001, 'batch_size': 16, 'optimizer_name': 'sgd', 'num_epochs': 10}


2026-01-24 06:40:04,012 - INFO - Epoch 1/10:   0%|          | 0/3062 [00:00<?, ?it/s]


2026-01-24 06:40:34,040 - INFO - Epoch 1/10:  32%|###2      | 987/3062 [00:30<01:03, 32.87it/s]


2026-01-24 06:40:48,820 - INFO - Epoch 1/10:  32%|###2      | 987/3062 [00:44<01:03, 32.87it/s]


2026-01-24 06:41:04,052 - INFO - Epoch 1/10:  66%|######6   | 2033/3062 [01:00<00:30, 34.04it/s]


...

2026-01-24 11:00:48,414 - INFO - Training completed in 17.59 minutes.


2026-01-24 11:00:49,400 - INFO - Training completed.



2026-01-24 11:00:49,406 - INFO - Best Model Config: {'dataset_name': 'fashion-mnist', 'model_name': 'resnet18', 'learning_rate': 0.001, 'batch_size': 16, 'optimizer_name': 'adam', 'num_epochs': 10, 'train_loss': [0.6037027287636996, 0.4075103825132182, 0.3484913632954981, 0.31037641931775345, 0.28513353522611473, 0.2660541118631793, 0.24737675910892243, 0.23526501587447896, 0.21971942734272554, 0.21017661972237833], 'train_accuracy': [0.776208360548661, 0.8497101567602874, 0.8727343239712606, 0.8866141410842586, 0.8957789026779882, 0.9009430111038537, 0.910291476159373, 0.9137410189418681, 0.9197624101894187, 0.9225179621162639], 'val_loss': [0.4900671697429248, 0.3306814578601292, 0.312796891863857, 0.2878682315775326, 0.25825168446344987, 0.24983851849181313, 0.23055743493565492, 0.21249712407588958, 0.19204095019293682, 0.19606179539965732], 'val_accuracy': [0.8324285714285714, 0.8787142857142857, 0.8815714285714286, 0.8907142857142857, 0.9041428571428571, 0.9078571428571428, 0.9147

2026-01-24 11:00:49,407 - INFO - Best Model Test Accuracy: 0.9299


2026-01-24 11:00:49,506 - INFO - Best model saved to: ./q1_a_experiments/fashion_mnist_best_model.pth


### Q1, b) Training SVM

In [24]:
numpy_mnist_dataset = None
numpy_fashion_mnist_dataset = None

In [25]:
def create_numpy_dataset(
    basedata: torch.utils.data.Dataset,
    transform: torchvision.transforms.Compose
    # n_augments: int = 1
) -> Tuple[np.array, np.array]:
    
    X_list = []
    y_list = []
    
    for img, label in tqdm(basedata, desc="Creating numpy dataset", file=tqdm_out, mininterval=30.0):
        img_np = transform(img).numpy().reshape(-1)
        X_list.append(img_np)
        y_list.append(int(label))
    
    X = np.stack(X_list)
    y = np.array(y_list)
    del X_list, y_list
    return X, y

In [26]:
def load_np_mnist_dataset() -> None:
    global numpy_mnist_dataset
    
    train_transform = T.Compose([
        T.ToPILImage(),
        T.RandomRotation(15),
        T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        T.ToTensor()
    ])

    test_transform = T.Compose([
        T.ToPILImage(),
        T.ToTensor()
    ])
    
    if numpy_mnist_dataset is not None:
        return
    
    train_data = create_numpy_dataset(
        basedata=mnist_dataset["train"],
        transform=train_transform
    )
    
    val_data = create_numpy_dataset(
        basedata=mnist_dataset["val"],
        transform=test_transform
    )
    
    test_data = create_numpy_dataset(
        basedata=mnist_dataset["test"],
        transform=test_transform
    )
    
    numpy_mnist_dataset = {
        "train": train_data,
        "val": val_data,
        "test": test_data
    }
    return


def load_np_fashion_mnist_dataset() -> None:
    global numpy_fashion_mnist_dataset
    
    train_transform = T.Compose([
        T.ToPILImage(),
        T.RandomRotation(15),
        T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        T.ToTensor()
    ])

    test_transform = T.Compose([
        T.ToPILImage(),
        T.ToTensor()
    ])
    
    if numpy_fashion_mnist_dataset is not None:
        return
    
    train_data = create_numpy_dataset(
        basedata=fashion_mnist_dataset["train"],
        transform=train_transform
    )
    
    val_data = create_numpy_dataset(
        basedata=fashion_mnist_dataset["val"],
        transform=test_transform
    )
    
    test_data = create_numpy_dataset(
        basedata=fashion_mnist_dataset["test"],
        transform=test_transform
    )
    numpy_fashion_mnist_dataset = {
        "train": train_data,
        "val": val_data,
        "test": test_data
    }
    return

def load_np_dataset() -> None:
    load_np_mnist_dataset()
    load_np_fashion_mnist_dataset()       

In [27]:
load_np_dataset()

2026-01-24 11:00:51,365 - INFO - Creating numpy dataset:   0%|          | 0/49000 [00:00<?, ?it/s]


2026-01-24 11:01:09,473 - INFO - Creating numpy dataset: 100%|##########| 49000/49000 [00:18<00:00, 2706.28it/s]


2026-01-24 11:01:09,582 - INFO - Creating numpy dataset:   0%|          | 0/7000 [00:00<?, ?it/s]


2026-01-24 11:01:10,989 - INFO - Creating numpy dataset: 100%|##########| 7000/7000 [00:01<00:00, 4980.24it/s]


2026-01-24 11:01:11,007 - INFO - Creating numpy dataset:   0%|          | 0/14000 [00:00<?, ?it/s]


...

2026-01-24 11:01:31,839 - INFO - Creating numpy dataset: 100%|##########| 49000/49000 [00:16<00:00, 2901.94it/s]


2026-01-24 11:01:31,962 - INFO - Creating numpy dataset:   0%|          | 0/7000 [00:00<?, ?it/s]


2026-01-24 11:01:33,338 - INFO - Creating numpy dataset: 100%|##########| 7000/7000 [00:01<00:00, 5089.19it/s]


2026-01-24 11:01:33,354 - INFO - Creating numpy dataset:   0%|          | 0/14000 [00:00<?, ?it/s]


2026-01-24 11:01:36,087 - INFO - Creating numpy dataset: 100%|##########| 14000/14000 [00:02<00:00, 5124.76it/s]


In [28]:
def save_svm_model_and_stats(
    model: svm.SVC,
    config: Dict[str, Any],
    training_stats: Dict[str, Any],
    base_dir: str    
) -> Tuple[str, str, str]:
    model_save_dir_name = '_'.join(f'{key}_{value}' for key, value in config.items())
    model_dir = os.path.join(base_dir, ''.join(model_save_dir_name))
    os.makedirs(model_dir, exist_ok=True) 
    
    model_path = os.path.join(model_dir, 'svm_model.pkl')
    joblib.dump(model, model_path)
    
    with open(os.path.join(model_dir, 'training_stats.json'), 'w') as f:
        json.dump(training_stats, f, indent=4)
    
    return (model_path, model_dir, base_dir)

In [29]:
def train_svm(config: Dict[str, Any], base_dir: str) -> Dict[str, Any]:
    # config has keys: dataset_name, kernel_name, kernel_params
    if config["dataset_name"] == "mnist":
        dataset = numpy_mnist_dataset
    elif config["dataset_name"] == "fashion_mnist":
        dataset = numpy_fashion_mnist_dataset
    else:
        raise ValueError(f"Unsupported dataset name: {config['dataset_name']}")
    
    X_train, y_train = dataset["train"]
    X_val, y_val = dataset["val"]
    X_test, y_test = dataset["test"]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    logger.info(f"Training SVM with config: {config}")
    
    if config["kernel_name"] == "poly":
        model = svm.SVC(
            kernel='poly',
            degree=config["kernel_params"].get("degree", 3),
            C=config["kernel_params"].get("C", 1.0),
            gamma=config["kernel_params"].get("gamma", 'scale'),
            random_state=SEED
        )
    elif config["kernel_name"] == "rbf":
        model = svm.SVC(
            kernel='rbf',
            C=config["kernel_params"].get("C", 1.0),
            gamma=config["kernel_params"].get("gamma", 'scale'),
            random_state=SEED
        )
    else:
        raise ValueError(f"Unsupported kernel name: {config['kernel_name']}")
    
    start_time = time.time()    
    model.fit(X_train, y_train)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    logger.info(
        f"SVM Training completed. "
        f"Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}, Test Acc: {test_accuracy:.4f}"
    )
    
    logger.info(f"SVM Training completed in {elapsed_time/60:.2f} minutes.")
    
    training_stats = {
        "train_accuracy": train_accuracy,
        "val_accuracy": val_accuracy,
        "test_accuracy": test_accuracy
    }
    
    model_bath, model_dir, _ = save_svm_model_and_stats(
        model,
        config,
        training_stats,
        base_dir=base_dir
    )
    
    training_stats["model_path"] = model_bath
    training_stats["model_dir"] = model_dir
    training_stats["base_dir"] = base_dir
    training_stats["training_time"] = elapsed_time # in seconds
    
    return training_stats

In [30]:
base_experiment_dir = './q1_b_experiments'
os.makedirs(base_experiment_dir, exist_ok=True)

mnist_search_space = {
    "dataset_name": ["mnist"],
    "kernel_name": ["poly", "rbf"],
    "kernel_params": {
        "poly": {
            "degree": [2, 3],
            "C": [0.1, 1.0],
            "gamma": ['scale', 'auto']
        },
        "rbf": {
            "C": [0.1, 1.0],
            "gamma": ['scale', 'auto']
        }
    }
}

results = []

for dataset_name in mnist_search_space["dataset_name"]:
    for kernel_name in mnist_search_space["kernel_name"]:
        if kernel_name == "poly":
            for degree in mnist_search_space["kernel_params"]["poly"]["degree"]:
                for C in mnist_search_space["kernel_params"]["poly"]["C"]:
                    for gamma in mnist_search_space["kernel_params"]["poly"]["gamma"]:
                        config = {
                            "dataset_name": dataset_name,
                            "kernel_name": kernel_name,
                            "kernel_params": {
                                "degree": degree,
                                "C": C,
                                "gamma": gamma
                            }
                        }
                        training_stats = train_svm(config, os.path.join(base_experiment_dir, 'mnist_svm_experiments'))
                        results.append({**config, **training_stats})
        elif kernel_name == "rbf":
            for C in mnist_search_space["kernel_params"]["rbf"]["C"]:
                for gamma in mnist_search_space["kernel_params"]["rbf"]["gamma"]:
                    config = {
                        "dataset_name": dataset_name,
                        "kernel_name": kernel_name,
                        "kernel_params": {
                            "C": C,
                            "gamma": gamma
                        }
                    }
                    training_stats = train_svm(config, os.path.join(base_experiment_dir, 'mnist_svm_experiments'))
                    results.append({**config, **training_stats})

# save the results to json file
with open(os.path.join(base_experiment_dir, 'mnist_svm_experiments_results.json'), 'w') as f:
    json.dump(results, f, indent=4)

# identify best model based on test accuracy
best_model = max(results, key=lambda x: x['test_accuracy'])
logger.info(f"Best SVM Model Config: {best_model}")
logger.info(f"Best SVM Model Test Accuracy: {best_model['test_accuracy']:.4f}")

# save best model to base_dir/mnist_svm_best_model.pkl
best_model_src_path = best_model['model_path']
best_model_dest_path = os.path.join(base_experiment_dir, 'mnist_svm_best_model.pkl')
shutil.copy(best_model_src_path, best_model_dest_path)
logger.info(f"Best SVM model saved to: {best_model_dest_path}")

2026-01-24 11:01:37,993 - INFO - Training SVM with config: {'dataset_name': 'mnist', 'kernel_name': 'poly', 'kernel_params': {'degree': 2, 'C': 0.1, 'gamma': 'scale'}}


2026-01-24 11:01:57,576 - INFO - SVM Training completed. Train Acc: 0.8635, Val Acc: 0.8797, Test Acc: 0.8737


2026-01-24 11:01:57,577 - INFO - SVM Training completed in 0.08 minutes.


2026-01-24 11:02:00,539 - INFO - Training SVM with config: {'dataset_name': 'mnist', 'kernel_name': 'poly', 'kernel_params': {'degree': 2, 'C': 0.1, 'gamma': 'auto'}}


2026-01-24 11:02:19,226 - INFO - SVM Training completed. Train Acc: 0.8606, Val Acc: 0.8771, Test Acc: 0.8701


...

2026-01-24 11:05:22,820 - INFO - SVM Training completed. Train Acc: 0.9661, Val Acc: 0.9591, Test Acc: 0.9599


2026-01-24 11:05:22,821 - INFO - SVM Training completed in 0.05 minutes.


2026-01-24 11:05:23,978 - INFO - Best SVM Model Config: {'dataset_name': 'mnist', 'kernel_name': 'rbf', 'kernel_params': {'C': 1.0, 'gamma': 'scale'}, 'train_accuracy': 0.9668367346938775, 'val_accuracy': 0.9598571428571429, 'test_accuracy': 0.9600714285714286, 'model_path': "./q1_b_experiments/mnist_svm_experiments/dataset_name_mnist_kernel_name_rbf_kernel_params_{'C': 1.0, 'gamma': 'scale'}/svm_model.pkl", 'model_dir': "./q1_b_experiments/mnist_svm_experiments/dataset_name_mnist_kernel_name_rbf_kernel_params_{'C': 1.0, 'gamma': 'scale'}", 'base_dir': './q1_b_experiments/mnist_svm_experiments', 'training_time': 3.2763125896453857}


2026-01-24 11:05:23,979 - INFO - Best SVM Model Test Accuracy: 0.9601


2026-01-24 11:05:24,270 - INFO - Best SVM model saved to: ./q1_b_experiments/mnist_svm_best_model.pkl


In [31]:
fashion_mnist_search_space = {
    "dataset_name": ["fashion_mnist"],
    "kernel_name": ["poly", "rbf"],
    "kernel_params": {
        "poly": {
            "degree": [2, 3],
            "C": [0.1, 1.0],
            "gamma": ['scale', 'auto']
        },
        "rbf": {
            "C": [0.1, 1.0],
            "gamma": ['scale', 'auto']
        }
    }
}

results = []

for dataset_name in fashion_mnist_search_space["dataset_name"]:
    for kernel_name in fashion_mnist_search_space["kernel_name"]:
        if kernel_name == "poly":
            for degree in fashion_mnist_search_space["kernel_params"]["poly"]["degree"]:
                for C in fashion_mnist_search_space["kernel_params"]["poly"]["C"]:
                    for gamma in fashion_mnist_search_space["kernel_params"]["poly"]["gamma"]:
                        config = {
                            "dataset_name": dataset_name,
                            "kernel_name": kernel_name,
                            "kernel_params": {
                                "degree": degree,
                                "C": C,
                                "gamma": gamma
                            }
                        }
                        training_stats = train_svm(config, os.path.join(base_experiment_dir, 'fashion_mnist_svm_experiments'))
                        results.append({**config, **training_stats})
        elif kernel_name == "rbf":
            for C in fashion_mnist_search_space["kernel_params"]["rbf"]["C"]:
                for gamma in fashion_mnist_search_space["kernel_params"]["rbf"]["gamma"]:
                    config = {
                        "dataset_name": dataset_name,
                        "kernel_name": kernel_name,
                        "kernel_params": {
                            "C": C,
                            "gamma": gamma
                        }
                    }
                    training_stats = train_svm(config, os.path.join(base_experiment_dir, 'fashion_mnist_svm_experiments'))
                    results.append({**config, **training_stats})

# save the results to json file
with open(os.path.join(base_experiment_dir, 'fashion_mnist_svm_experiments_results.json'), 'w') as f:
    json.dump(results, f, indent=4)

# identify best model based on test accuracy
best_model = max(results, key=lambda x: x['test_accuracy'])
logger.info(f"Best SVM Model Config: {best_model}")
logger.info(f"Best SVM Model Test Accuracy: {best_model['test_accuracy']:.4f}")

# save best model to base_dir/fashion_mnist_svm_best_model.pkl
best_model_src_path = best_model['model_path']
best_model_dest_path = os.path.join(base_experiment_dir, 'fashion_mnist_svm_best_model.pkl')
shutil.copy(best_model_src_path, best_model_dest_path)
logger.info(f"Best SVM model saved to: {best_model_dest_path}")

2026-01-24 11:05:25,033 - INFO - Training SVM with config: {'dataset_name': 'fashion_mnist', 'kernel_name': 'poly', 'kernel_params': {'degree': 2, 'C': 0.1, 'gamma': 'scale'}}


2026-01-24 11:05:37,715 - INFO - SVM Training completed. Train Acc: 0.7509, Val Acc: 0.7516, Test Acc: 0.7516


2026-01-24 11:05:37,716 - INFO - SVM Training completed in 0.06 minutes.


2026-01-24 11:05:39,935 - INFO - Training SVM with config: {'dataset_name': 'fashion_mnist', 'kernel_name': 'poly', 'kernel_params': {'degree': 2, 'C': 0.1, 'gamma': 'auto'}}


2026-01-24 11:05:52,475 - INFO - SVM Training completed. Train Acc: 0.7509, Val Acc: 0.7516, Test Acc: 0.7516


...

2026-01-24 11:07:59,379 - INFO - SVM Training completed. Train Acc: 0.8716, Val Acc: 0.8293, Test Acc: 0.8244


2026-01-24 11:07:59,380 - INFO - SVM Training completed in 0.05 minutes.


2026-01-24 11:08:00,321 - INFO - Best SVM Model Config: {'dataset_name': 'fashion_mnist', 'kernel_name': 'rbf', 'kernel_params': {'C': 1.0, 'gamma': 'scale'}, 'train_accuracy': 0.8715714285714286, 'val_accuracy': 0.8292857142857143, 'test_accuracy': 0.8243571428571429, 'model_path': "./q1_b_experiments/fashion_mnist_svm_experiments/dataset_name_fashion_mnist_kernel_name_rbf_kernel_params_{'C': 1.0, 'gamma': 'scale'}/svm_model.pkl", 'model_dir': "./q1_b_experiments/fashion_mnist_svm_experiments/dataset_name_fashion_mnist_kernel_name_rbf_kernel_params_{'C': 1.0, 'gamma': 'scale'}", 'base_dir': './q1_b_experiments/fashion_mnist_svm_experiments', 'training_time': 3.015918016433716}


2026-01-24 11:08:00,323 - INFO - Best SVM Model Test Accuracy: 0.8244


2026-01-24 11:08:00,561 - INFO - Best SVM model saved to: ./q1_b_experiments/fashion_mnist_svm_best_model.pkl


### Q2) Performance comparison on Fashion-MNIST

In [22]:
# Gpu must be present for this code to run
if not torch.cuda.is_available():
    logger.error("GPU is not available. Exiting the program.")
    exit(1)

In [23]:
base_experiment_dir = './q2_experiments'
os.makedirs(base_experiment_dir, exist_ok=True)

search_space = {
    "dataset_name": "fashion-mnist",
    "model_name": ["resnet18", "resnet32", "resnet50"],
    "learning_rate": [0.001],
    "batch_size": [16],
    "optimizer_name": ["sgd", "adam"],
    "num_epochs": [5],
    "device": ['cpu', 'cuda']
}

results = []


for device_type in search_space["device"]:
    for model_name in search_space["model_name"]:
        for optimizer_name in search_space["optimizer_name"]:
            config = {
                "dataset_name": search_space["dataset_name"],
                "model_name": model_name,
                "learning_rate": search_space["learning_rate"][0],
                "batch_size": search_space["batch_size"][0],
                "optimizer_name": optimizer_name,
                "num_epochs": search_space["num_epochs"][0],
                "device": device_type
            }
            device = torch.device(device_type)
            logger.info(f"Training with config: {config} on device: {device}")
            return_data = train_model(
                config, os.path.join(base_experiment_dir, f'{device_type}_experiments'),
                device=device
            )
            training_stats = {}
            logger.info(f"Training completed.\n")
            results.extend(return_data)


# save the results to json file
with open(os.path.join(base_experiment_dir, 'device_comparison_experiments_results.json'), 'w') as f:
    json.dump(results, f, indent=4)

# calculate FLOPs for each configuration
for result in results:
    model_name = result["model_name"]
    if model_name == "resnet18":
        model = get_resnet18_model().to('cpu')
    elif model_name == "resnet32":
        model = get_resnet32_model().to('cpu')
    elif model_name == "resnet50":
        model = get_resnet50_model().to('cpu')
    else:
        raise ValueError(f"Unsupported model name: {model_name}")
    model.to(result["device"])
    model.eval()
    dummy_input = torch.randn(1, 3, 224, 224).to(result["device"])
    total_flops = FlopCountAnalysis(model, dummy_input)
    result['flops'] = total_flops.total()
    print(f"Model: {model_name}, Device: {result['device']}, FLOPs: {total_flops.total()}")

# save the results to json file
with open(os.path.join(base_experiment_dir, 'device_comparison_experiments_results.json'), 'w') as f:
    json.dump(results, f, indent=4)

2026-01-24 06:05:58,067 - INFO - Training with config: {'dataset_name': 'fashion-mnist', 'model_name': 'resnet18', 'learning_rate': 0.001, 'batch_size': 16, 'optimizer_name': 'sgd', 'num_epochs': 5, 'device': 'cpu'} on device: cpu


2026-01-24 06:05:58,954 - INFO - Epoch 1/5:   0%|          | 0/3062 [00:00<?, ?it/s]


2026-01-24 06:06:29,029 - INFO - Epoch 1/5:   2%|1         | 54/3062 [00:30<27:55,  1.80it/s]


2026-01-24 06:06:42,543 - INFO - Epoch 1/5:   2%|1         | 54/3062 [00:43<27:55,  1.80it/s]


2026-01-24 06:06:59,107 - INFO - Epoch 1/5:  11%|#         | 336/3062 [01:00<07:15,  6.25it/s]


...

/home/b23cs1037/MLOps-Laksh-B23CS1037/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/b23cs1037/MLOps-Laksh-B23CS1037/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model: resnet50, Device: cuda, FLOPs: 4109485056


Unsupported operator aten::max_pool2d encountered 1 time(s)


Unsupported operator aten::add_ encountered 16 time(s)


Model: resnet50, Device: cuda, FLOPs: 4109485056


In [2]:
def save_exhaustive_q1a_plots(results_file, exp_folder):
    if not os.path.exists(results_file): return
    with open(results_file, 'r') as f:
        data = json.load(f)
    
    df = pd.DataFrame(data)
    
    # FIX: If accuracy/loss columns are lists, extract the last element (final epoch)
    # This prevents the "unsupported operand type(s) for -: 'list' and 'float'" error
    for col in ['train_accuracy', 'test_accuracy', 'train_loss', 'test_loss', 'val_accuracy', 'val_loss']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: x[-1] if isinstance(x, list) else x)
    
    plot_dir = os.path.join(exp_folder, 'plots')
    os.makedirs(plot_dir, exist_ok=True)
    sns.set_theme(style="whitegrid", palette="viridis")

    # Now this math will work because columns are floats
    df['generalization_gap'] = df['train_accuracy'] - df['test_accuracy']

    # --- 1. ACCURACY HEATMAP ---
    plt.figure(figsize=(14, 8))
    pivot_acc = df.pivot_table(index=['model_name', 'optimizer_name'], 
                               columns=['learning_rate', 'batch_size'], 
                               values='test_accuracy')
    sns.heatmap(pivot_acc, annot=True, fmt=".4f", cmap="YlGnBu")
    plt.title(f"Test Accuracy Matrix (Final Epoch)")
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'accuracy_matrix.png'))

    # --- 2. OVERFITTING ANALYSIS ---
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='model_name', y='generalization_gap', hue='optimizer_name')
    plt.title("Generalization Gap (Final Epoch)")
    plt.savefig(os.path.join(plot_dir, 'overfitting_analysis.png'))

    # --- 3. PARAMETER FACETS ---
    g = sns.catplot(data=df, kind="bar", x="model_name", y="test_accuracy", 
                    hue="optimizer_name", col="batch_size", row="learning_rate")
    for ax in g.axes.flat:
        for container in ax.containers:
            ax.bar_label(container, fmt='%.3f', padding=3)
    plt.savefig(os.path.join(plot_dir, 'full_parameter_grid.png'))
    plt.close('all')
    
    # Save the cleaned-up matrix for your report
    pivot_acc.to_csv(os.path.join(plot_dir, 'full_comparison_matrix.csv'))

def save_exhaustive_q1b_plots(results_file, exp_folder):
    """SVM Analysis: Kernel sensitivity, degree impacts, and C scaling."""
    with open(results_file, 'r') as f:
        data = json.load(f)
    
    rows = []
    for d in data:
        row = {k: v for k, v in d.items() if k != 'kernel_params'}
        row.update(d['kernel_params'])
        rows.append(row)
    df = pd.DataFrame(rows)
    
    plot_dir = os.path.join(exp_folder, 'plots')
    os.makedirs(plot_dir, exist_ok=True)

    # --- 1. SVM CROSS-COMPARISON TABLE ---
    svm_matrix = df.pivot_table(
        index=['kernel_name', 'degree'], 
        columns=['C', 'gamma'], 
        values='test_accuracy'
    ).fillna('N/A')
    svm_matrix.to_csv(os.path.join(plot_dir, 'svm_param_matrix.csv'))

    # --- 2. RBF & POLY HEATMAPS ---
    for kernel in ['rbf', 'poly']:
        k_df = df[df['kernel_name'] == kernel]
        if not k_df.empty:
            plt.figure(figsize=(10, 6))
            index_val = 'C' if kernel == 'rbf' else 'degree'
            col_val = 'gamma' if kernel == 'rbf' else 'C'
            pivot_k = k_df.pivot_table(index=index_val, columns=col_val, values='test_accuracy')
            sns.heatmap(pivot_k, annot=True, fmt=".4f", cmap="magma")
            plt.title(f"SVM {kernel.upper()} Sensitivity: {index_val} vs {col_val}")
            plt.savefig(os.path.join(plot_dir, f'svm_{kernel}_heatmap.png'))

    # --- 3. KERNEL PERFORMANCE COMPARISON ---
    plt.figure(figsize=(10, 6))
    ax = sns.pointplot(data=df, x='C', y='test_accuracy', hue='kernel_name', markers=['o', 's'], linestyles=['-', '--'])
    plt.title("Test Accuracy Trend across Regularization (C)")
    plt.savefig(os.path.join(plot_dir, 'svm_accuracy_trends.png'))
    plt.close('all')

def save_exhaustive_q2_plots(results_file, exp_folder):
    """Hardware Analysis: Throughput, Latency, and Cost-Efficiency."""
    with open(results_file, 'r') as f:
        df = pd.DataFrame(json.load(f))
    
    plot_dir = os.path.join(exp_folder, 'plots')
    os.makedirs(plot_dir, exist_ok=True)

    # --- 1. HARDWARE EFFICIENCY MATRIX ---
    # Calculates Accuracy per FLOP and Time per FLOP
    df['acc_per_sec'] = df['test_accuracy'] / df['training_time']
    df['time_per_megaflop'] = df['training_time'] / (df['flops'] / 1e6)
    df.to_csv(os.path.join(plot_dir, 'hardware_efficiency_metrics.csv'))

    # --- 2. THE HARDWARE SPEEDUP DASHBOARD ---
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))

    # Speedup: Bar chart with labels
    sns.barplot(data=df, x='model_name', y='training_time', hue='device', ax=axes[0])
    axes[0].set_yscale('log')
    axes[0].set_title("Training Latency (Seconds, Log Scale)")
    for c in axes[0].containers: axes[0].bar_label(c, fmt='%.1f')

    # Throughput vs Accuracy: Scatter
    sns.scatterplot(data=df, x='flops', y='test_accuracy', hue='device', size='acc_per_sec', 
                    sizes=(100, 1000), style='model_name', ax=axes[1])
    axes[1].set_title("Compute Intensity (FLOPs) vs Accuracy (Bubble Size = Efficiency)")
    
    plt.savefig(os.path.join(plot_dir, 'hardware_performance_dashboard.png'))

    # --- 3. GPU UTILIZATION VALUE ---
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=df, x='model_name', y='acc_per_sec', hue='device')
    for c in ax.containers: ax.bar_label(c, fmt='%.4f')
    plt.title("Return on Compute: Accuracy gained per second of training")
    plt.savefig(os.path.join(plot_dir, 'return_on_compute.png'))
    plt.close('all')

In [3]:
save_exhaustive_q1a_plots('./q1_a_experiments/mnist_experiments_results.json', './q1_a_experiments')
save_exhaustive_q1b_plots('./q1_b_experiments/mnist_svm_experiments_results.json', './q1_b_experiments')
save_exhaustive_q2_plots('./q2_experiments/device_comparison_experiments_results.json', './q2_experiments')